In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# OVERVİEW

---

### Hey there! I want to show my study about crimes in boston dataset. Before analyzing the data, I want to fix the shortcomings of dataset. You can see it below. I describe this step as research.

                                                                *   Upload data
                                                                *   Looking at general information (type etc.)
                                                                *   Filling in blank values

<img src="https://media1.tenor.com/images/55f2496603cfa0243089afc587956eaa/tenor.gif?itemid=9938974" width="273" height="273" alt="Research GIF - Research GIFs" style="max-width: 273px; background-color: rgb(63, 63, 63);">

## **Let's do some research!**

In [ ]:
import pandas as pd

data= pd.read_csv('../input/crimes-in-boston/crime.csv',encoding="iso-8859-1")

data.info()

In [ ]:
data.head()

## I rearranged the lat AND long values according to the values in the location column and arranged the dates so that I could draw a time series. then I filled in the blanks.


In [ ]:
data[['Lat-1','Long-1']]=data['Location'].str.split(",", expand = True)
data['Lat'] = data['Lat-1'].str[1:]
data['Long'] = data['Long-1'].str[:-1]

In [ ]:
data['Lat']=data['Lat'].astype(float)
data['Long']=data['Long'].astype(float)

In [ ]:
import datetime
data['OCCURRED_ON_DATE']=pd.to_datetime(data['OCCURRED_ON_DATE'])
data['Date']=data['OCCURRED_ON_DATE'].dt.strftime('%Y-%m-%d')
data['Date']=pd.to_datetime(data['Date'])
data['MONTH'] = data.Date.dt.strftime("%m")

In [ ]:
data.isna().any()[lambda x: x] # I wanted to determine which columns have NaN values, then I will examine these columns one by one.

In [ ]:
data['DISTRICT'].value_counts() #I will write 'unclear' in nan places.

In [ ]:
data['DISTRICT'].fillna('unclear',inplace=True)

In [ ]:
data['SHOOTING'].value_counts() # I understand that they wanted to label it as Yes and No, so I'll write N to Nan values.


In [ ]:
data['SHOOTING'].fillna('N',inplace=True)

In [ ]:
data['UCR_PART'].value_counts() # I will call those who are NaN "uncertain".

In [ ]:
data['UCR_PART'].fillna('uncertain',inplace=True)

In [ ]:
data['STREET'].value_counts() # I will call those who are NaN "uncertain".

In [ ]:
data['STREET'].fillna('uncertain',inplace=True)

## All of them is filled. Great!

## LET'S DIVE IN!

In [ ]:
data.describe() #there is something wrong with lat and long. 

In [ ]:
(data.groupby(["Lat","Long"]).count()[['INCIDENT_NUMBER']]).reset_index()

# There are question patterns where we can reach general information about data. WHO? WHERE? WHEN? WHY? HOW?

# WHO?                        

 * Which of the offense code group is more?

# WHERE?  (What can you say about the distribution of different offenses over the city?)

 * In which district more?
 * Which street is more?
 * Which reporting area is the most?

# WHEN?

 * Has it increased over the years?
 * Is it variable by day and month?
 * Does the clock affect? Is it more evening?

# HOW?

 * How does it affect whether it is shooting or not?

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
x=pd.pivot_table(data, index='OFFENSE_CODE_GROUP',values="INCIDENT_NUMBER", aggfunc="count", fill_value=0)
x=x.reset_index()
x = x.reindex(x.sort_values(by=['INCIDENT_NUMBER'], ascending=False).index)
x=x[:10]
fig_dims = (8, 7)
fig, ax = plt.subplots(figsize=fig_dims)
ax = sns.barplot(x="INCIDENT_NUMBER",y="OFFENSE_CODE_GROUP", data=x)

#We can see that the most of the crimes occured named Motor Vehicle Accident Response. Larceny and medical assistance follow it.

# 1. How has crime changed over the years?

 * While there has been an increase over the years since 2015, we see that it decreased in 2018.
 * When we look at the year as a month; There were obvious decreases in the 6th month of 2015 and the 9th month of 2018.
 * There is no extreme difference according to the days. They are all close to each other.
 * The highest hours in the evening according to the hours. at least between 3-5 at night.


## When we say all of above, we could be wrong. Because in some years, some of months are missing.So,we cannot reach the conclusion just by looking at these.




In [ ]:
x=pd.pivot_table(data, index='YEAR',values="INCIDENT_NUMBER", aggfunc="count", fill_value=0)
x=x.reset_index()
ax = sns.barplot(x="YEAR", y="INCIDENT_NUMBER", data=x)


In [ ]:
import altair as alt
x=pd.pivot_table(data, index='MONTH',values="INCIDENT_NUMBER", aggfunc="count", fill_value=0)
x=x.reset_index()
ax = sns.barplot(x="MONTH", y="INCIDENT_NUMBER", data=x)




In [ ]:
import altair as alt
x=pd.pivot_table(data, index='DAY_OF_WEEK',values="INCIDENT_NUMBER", aggfunc="count", fill_value=0)
x=x.reset_index()
ax = sns.barplot(x="DAY_OF_WEEK", y="INCIDENT_NUMBER", data=x)



In [ ]:
x=pd.pivot_table(data, index='HOUR',values="INCIDENT_NUMBER", aggfunc="count", fill_value=0)
x=x.reset_index()

ax = sns.barplot(x="HOUR", y="INCIDENT_NUMBER", data=x)



# Now, we can look average of crime in months,year...

In [ ]:
data.head()
data['Day']=data.OCCURRED_ON_DATE.dt.day
x=pd.pivot_table(data, index=['YEAR','MONTH','Day'],values="INCIDENT_NUMBER", aggfunc='count', fill_value=0)
x=x.reset_index()
Y=pd.pivot_table(x, index=['YEAR'],values="INCIDENT_NUMBER", aggfunc=np.mean, fill_value=0)
Y=Y.reset_index()
Y.INCIDENT_NUMBER.round(0)


a=pd.pivot_table(data, index=['YEAR','MONTH','Day'],values="INCIDENT_NUMBER", aggfunc='count', fill_value=0)
a=a.reset_index()
b=pd.pivot_table(a, index=['MONTH'],values="INCIDENT_NUMBER", aggfunc=np.mean, fill_value=0)
b=b.reset_index()
b.INCIDENT_NUMBER.round(0)

c=pd.pivot_table(data, index=['YEAR','MONTH','Day','DAY_OF_WEEK'],values="INCIDENT_NUMBER", aggfunc='count', fill_value=0)
c=c.reset_index()
d=pd.pivot_table(c, index=['DAY_OF_WEEK'],values="INCIDENT_NUMBER", aggfunc=np.mean, fill_value=0)
d=d.reset_index()
d.INCIDENT_NUMBER.round(0)

a4_dims = (20, 5)
fig, (ax1, ax2,ax3) = plt.subplots(ncols=3, sharey=True,figsize=a4_dims)

ax = sns.barplot(x="YEAR", y="INCIDENT_NUMBER", data=Y,ax=ax1,color='lightblue')

ax = sns.barplot(x="MONTH", y="INCIDENT_NUMBER", data=b,ax=ax2,color='lightgreen')

ax = sns.barplot(x="DAY_OF_WEEK", y="INCIDENT_NUMBER", data=d,ax=ax3,color='lightseagreen')


Here, my comment is;

## Yearly

### * I can not say obvious comment. But, I want to look at deeply.Maybe we can do statistical test for this.

## Monthly

### * We can see that crimes increase towards May and June. But we cannot say that there is an increase in the whole summer. Because, our average daily number of thefts decreased in the months after June.

## Day of Week

### * When we look at the average daily numbers of thefts on the chart, there is a clear decrease on Sunday. But no interpretation can be made for other days like this.




# GENERAL TIME LINE

### We can see that the average daily crimes in months rise during the summer months. Especially in the summer of 2017, there is a huge increase in numbers. and we can see cycles. In the winter months, it increases less in summer and decreases again towards the end of the year.

In [ ]:
x=pd.pivot_table(data, index=['Date','YEAR','MONTH','Day'],values="INCIDENT_NUMBER", aggfunc="count", fill_value=0)
x=x.reset_index()

x['MONTH'] = x['MONTH'].astype(str)
x['YEAR']=x['YEAR'].astype(str)
x['YEARMONTH'] = x['YEAR'].str.cat(x['MONTH'],sep="")

y=pd.pivot_table(x, index=['YEARMONTH'],values="INCIDENT_NUMBER", aggfunc="mean", fill_value=0)
y=y.reset_index()
y['YEARMONTH']=y['YEARMONTH'].astype(str)
y = y.reindex(y.sort_values(by=['YEARMONTH'], ascending=[True]).index)

fig, ax = plt.subplots()
fig.set_size_inches(15, 5)
sns.set(style="darkgrid")
g=sns.lineplot(x="YEARMONTH", y="INCIDENT_NUMBER",data=y)
plt.xticks(rotation=70)
plt.tight_layout()

# RANDOM ANALYSIS

### I wanted to apply a pareto analysis to offence groups. In this way, if a measure is to be taken. By giving priority to the A group that has the most impact, an 80% efficiency can be achieved.

### Group A came out as follows:

*   Motor Vehicle Accident Response
*   Larceny
*   Medical Assistance
*   Investigate Person
*   Other
*   Drug Violation
*   Simple Assault
*   Vandalism
*   Verbal Disputes
*   Towed
*   Investigate Property
*   Larceny From Motor Vehicle
*   Property Lost
*   Warrant Arrests
*   Aggravated Assault
*   Violations


In [ ]:
abc=pd.pivot_table(data, index=['OFFENSE_CODE_GROUP'],values="INCIDENT_NUMBER",  aggfunc='count', fill_value=0)
abc=abc.reset_index()
abc = abc.reindex(abc.sort_values(by='INCIDENT_NUMBER', ascending=False).index)
abc['cumsum'] = abc['INCIDENT_NUMBER'].cumsum()
abc['sum'] = abc['INCIDENT_NUMBER'].sum()
abc['percentage'] =  (abc['cumsum']/abc['sum'])*100

def ABC_segmentation(perc):
   
     if perc > 0 and perc < 80:
        return 'A'
     elif perc >= 80 and perc < 95:
        return 'B'
     elif perc >= 95:
        return 'C'



abc['clustering'] = abc['percentage'].apply(ABC_segmentation)
abc[abc['clustering']=='A']

# Change of offense code statuses by years

### We know that in the crime data, some of the months are missing. After, we look at the data, it can be seen in year 2016 and 2017 all of the months avaible. So, I found the increase percentages of offense codes between 2016-2017 values. There are extremely increasing or decreasing varieties here. Some of them have a high effect even if their numbers are small. For instance, there is obvious changing  on Prostituon in 2017. It has changed in the direction of an increase of 40%.

In [ ]:
data.groupby("MONTH")["YEAR"].unique()

In [ ]:
x=pd.pivot_table(data, index=['OFFENSE_CODE_GROUP','YEAR','MONTH','Day'],values="INCIDENT_NUMBER", aggfunc="count", fill_value=0)
x=x.reset_index()
Y=pd.pivot_table(x, index='OFFENSE_CODE_GROUP',columns='YEAR',values="INCIDENT_NUMBER", aggfunc=np.mean, fill_value=0)
Y=Y.reset_index()
Y.columns=['OFFENSE_CODE_GROUP','YIL2015','YIL2016','YIL2017','YIL2018']
Y['DEGISIM']=((Y['YIL2017']-Y['YIL2016'])/Y['YIL2016'])*100
Y=Y.replace([np.inf,-np.inf], np.nan)
Y=Y.dropna()
Y = Y.reindex(Y.sort_values(by="DEGISIM", ascending=False).index)
fig_dims = (10, 13)
fig, ax = plt.subplots(figsize=fig_dims)
ax = sns.barplot(x="DEGISIM", y="OFFENSE_CODE_GROUP", data=Y)

# HYPOTHESIS - IS THERE A DIFFERENCE  IN 2016 AND 2017?

## I have said that in the above analyzes, statistical methods can be used to understand whether there is a difference over the years. I wanted to run a hypothesis test to see if there was a difference in daily crime numbers between 2016 and 2017.

## Method: We can use MANN WHITNEY-U test for compare  of 2 years.

### H0: μd = 0 
### H1: μd ≠ 0

### 1.first we should seperate years 2016 and 2017

In [ ]:
hypothesis=data[(data['YEAR']==2016) | (data['YEAR']==2017)]

### 2. We should found the crime number in year-month-day format

In [ ]:
boxplot=pd.pivot_table(hypothesis, index=['YEAR','MONTH','Day'],values="INCIDENT_NUMBER", aggfunc="count", fill_value=0)
boxplot=boxplot.reset_index()

### 3. We can draw a boxplot for see the general situation

In [ ]:
ax = sns.boxplot(x="YEAR", y="INCIDENT_NUMBER", data=boxplot)

### 4. We should do MANN WHITNEY-U to testing the hypthesis

In [ ]:
from scipy.stats import mannwhitneyu
test=pd.pivot_table(hypothesis, index=['Day'],columns='YEAR',values="INCIDENT_NUMBER", aggfunc="count", fill_value=0)
test.columns=['YIL2016','YIL2017']
stat, p = mannwhitneyu(test['YIL2016'], test['YIL2017'])
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Same distribution (fail to reject H0)')
else:
	print('Different distribution (reject H0)')

### We find p value <0,05. So, the findings are statistically significant! One can reject the H0 hypothesis in support of the alternative.So statistically, there is a difference between years 2016 and 2017 for daily crimes.


### Then, we look boxplot above and in year 2017 there are less outlier then year 2016 and the median of crime is higher too. We can say that crimes increased in 2017.

<img alt="The Simpsons Crime GIF - Find &amp; Share on GIPHY" class="n3VNCb" src="https://media1.giphy.com/media/l2Je5OTIcc8fqrjs4/giphy.gif" data-noaft="1" jsname="HiaYvf" jsaction="load:XAeZkd,gvK6lb;" style="width: 428px; height: 322.783px; margin: 0px;">

# 2. What can you say about the distribution of different offenses over the city?



In [ ]:
x=pd.pivot_table(data, index='DISTRICT',values="INCIDENT_NUMBER", aggfunc="count", fill_value=0)
x=x.reset_index()
x = x.reindex(x.sort_values(by=['INCIDENT_NUMBER'], ascending=False).index)
ax = sns.barplot(x="DISTRICT", y="INCIDENT_NUMBER", data=x)
#We can see that the crimes are concentrated in the B2 district.

In [ ]:
x=pd.pivot_table(data, index='STREET',values="INCIDENT_NUMBER", aggfunc="count", fill_value=0)
x=x.reset_index()
x = x.reindex(x.sort_values(by=['INCIDENT_NUMBER'], ascending=False).index)
x.head()
#We can see that the crimes are concentrated in WASHINGTON ST. Crimes with an uncertain values follow it.

In [ ]:
x=pd.pivot_table(data, index='REPORTING_AREA',values="INCIDENT_NUMBER", aggfunc="count", fill_value=0)
x=x.reset_index()
x = x.reindex(x.sort_values(by=['INCIDENT_NUMBER'], ascending=False).index)
x.head()
#There are records with an empty reporting area section.

In [ ]:
x=pd.pivot_table(data, index=['DISTRICT','OFFENSE_CODE_GROUP'],values="INCIDENT_NUMBER", aggfunc="count", fill_value=0)
x=x.reset_index()
x['D-O'] = x['DISTRICT'].str.cat(x['OFFENSE_CODE_GROUP'],sep="-")
x = x.reindex(x.sort_values(by=['INCIDENT_NUMBER','DISTRICT'], ascending=False).index)
xvalue=x[:5]
fig_dims = (5, 5)
fig, ax = plt.subplots(figsize=fig_dims)
ax = sns.barplot(x="INCIDENT_NUMBER", y="D-O", data=xvalue)

## When we look at the top 30, the types of crimes in the B2, B3, C11 and D4 regions draw our attention.

In [ ]:
pivfordis=pd.pivot_table(data, index=['DISTRICT','OFFENSE_CODE_GROUP'],values="INCIDENT_NUMBER", aggfunc="count", fill_value=0)
pivfordis=pivfordis.reset_index()

In [ ]:
pivfordisB2=pivfordis[(pivfordis['DISTRICT']== 'B2') ]
pivfordisB2['D-O'] = pivfordisB2['DISTRICT'].str.cat(pivfordisB2['OFFENSE_CODE_GROUP'],sep="-")
pivfordisB2 = pivfordisB2.reindex(pivfordisB2.sort_values(by=['DISTRICT','INCIDENT_NUMBER'], ascending=False).index)
xvalue=pivfordisB2[:5]
fig_dims = (5, 5)
fig, ax = plt.subplots(figsize=fig_dims)
ax = sns.barplot(x="INCIDENT_NUMBER", y="D-O", data=xvalue)

In [ ]:
pivfordisC11=pivfordis[(pivfordis['DISTRICT']== 'C11') ]
pivfordisC11['D-O'] = pivfordisC11['DISTRICT'].str.cat(pivfordisC11['OFFENSE_CODE_GROUP'],sep="-")
pivfordisC11 = pivfordisC11.reindex(pivfordisC11.sort_values(by=['DISTRICT','INCIDENT_NUMBER'], ascending=False).index)
xvalue=pivfordisC11[:5]

fig_dims = (5, 5)
fig, ax = plt.subplots(figsize=fig_dims)
ax = sns.barplot(x="INCIDENT_NUMBER", y="D-O", data=xvalue)

In [ ]:
pivfordisD4=pivfordis[(pivfordis['DISTRICT']== 'D4') ]
pivfordisD4['D-O'] = pivfordisD4['DISTRICT'].str.cat(pivfordisD4['OFFENSE_CODE_GROUP'],sep="-")
pivfordisD4 = pivfordisD4.reindex(pivfordisD4.sort_values(by=['DISTRICT','INCIDENT_NUMBER'], ascending=False).index)
xvalue=pivfordisD4[:5]
fig_dims = (5, 5)
fig, ax = plt.subplots(figsize=fig_dims)
ax = sns.barplot(x="INCIDENT_NUMBER", y="D-O", data=xvalue)

In [ ]:
pivfordisB3=pivfordis[(pivfordis['DISTRICT']== 'B3') ]
pivfordisB3['D-O'] = pivfordisB3['DISTRICT'].str.cat(pivfordisB3['OFFENSE_CODE_GROUP'],sep="-")
pivfordisB3 = pivfordisB3.reindex(pivfordisB3.sort_values(by=['DISTRICT','INCIDENT_NUMBER'], ascending=False).index)
xvalue=pivfordisB3[:5]
fig_dims = (5, 5)
fig, ax = plt.subplots(figsize=fig_dims)
ax = sns.barplot(x="INCIDENT_NUMBER", y="D-O", data=xvalue)

### When we look at the selected regions, we can say that high crimes are generally similar. these:
### * Motor Vehicle Accident Response
### * Verbal Disputes
### * Medical Asistanse

### Only for D4 region **larceny** comes first.

# If you noticed, we also detected these crimes with pareto analysis.

### Let's try some more mapping methods.

<img src="https://media1.tenor.com/images/0c7d7c3c22d522674057187eae35cd22/tenor.gif?itemid=13892258" width="273" height="174.72" alt="Lost Kermit GIF - Lost Kermit Map GIFs" style="max-width: 273px; background-color: rgb(63, 63, 63);">


### (-1,-1) and (0,0) coordinates coincide with the middle of the ocean. That's why I drop values.



In [ ]:
(data.groupby(["Lat","Long"]).count()[['INCIDENT_NUMBER']]).reset_index()

In [ ]:
data_folium=data.drop(data[(data['Lat']==-1) & (data['Long']==-1)].index)
data_folium1=data_folium.drop(data_folium[(data_folium['Lat']==0) & (data_folium['Long']==0)].index)

In [ ]:
import folium
from folium import plugins
from folium.plugins import HeatMap
crime_map = folium.Map(location=[42.3125,-71.0875], 
                       tiles = "Stamen Terrain",
                      zoom_start = 11)
df_drop=data_folium1.dropna(subset=['Lat', 'Long', 'DISTRICT'])
# Add data for heatmp 
data_heatmap = df_drop[df_drop["YEAR"]==2016]
data_heatmap = df_drop[['Lat','Long']]
data_heatmap = df_drop.dropna(axis=0, subset=['Lat','Long'])
data_heatmap = [[row['Lat'],row['Long']] for index, row in data_heatmap.iterrows()]
HeatMap(data_heatmap, radius=10).add_to(crime_map)

# Plot!
crime_map

### When looked separately at the Engine Vehicle Accident Response, it seems that it has spread everywhere.

### The intensity of crime is higher on the main streets.But,the cops may have deliberately written the main streets. 

# 3.Is it possible to predict where or when a crime will be committed?

### When a detailed forecast is required, for example, hourly work will become increasingly difficult. Because there are manual entries in the data, the margin of error can be high.

### In general, As a first step, I think that data with a lot of categorical variables can be started with logistic regression and then looked at in detail. We can use month, day, street variables. In this way, a simple estimate can be made as a first step.

# THANK YOU!